In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.interpolate import interp1d

In [38]:
raw_input = pd.read_csv('../../Load Profile - Max Load.csv', header=None, skiprows = 12, nrows=1)

raw_input = raw_input.dropna(axis='columns')
raw_input = raw_input.drop([6, 7, 57], axis='columns')
hourly_load = []

for i in raw_input.iloc[0]:
    hourly_load.append(i)

In [82]:
hourly_load

[726,
 726,
 726,
 726,
 726,
 726,
 726,
 726,
 '9,671',
 '9,671',
 '9,671',
 '9,671',
 '9,671',
 '9,671',
 '9,671',
 '9,671',
 '9,671',
 '9,671',
 '9,671',
 '9,671',
 726,
 726,
 726,
 726]

In [74]:
K = 24 #hours?
days = 365

PVsize = 1700 #Watts
pvData = pd.read_csv('pvwatts_hourly.csv', skiprows = 17) #From PVwatts Calculator
DC_output = pvData['DC Array Output (W)'] #DC Array Output (W)
daily_output = DC_output.iloc[:-1].values #drop final totals row
daily_output = daily_output.reshape((24, 365))

pvDaysFinal = interp1d((np.linspace(1, 24, 24)), daily_output, axis=0)
pvo = pvDaysFinal(np.linspace(1, 24, 24))

demand_df = pd.read_csv('../../Load Profile - Max Load.csv') # Max load csv
demand = demand_df.iloc[8,8:55] # Indoor Lighting
# drop nan values --- I think this is what line 23 in matlab code is doing
demand = demand.dropna()
# demand1 variable in matlab code
load_df = pd.DataFrame(np.tile(demand.values, (days,1)).T, columns=[np.linspace(1,days,days)])

In [81]:
type(demand.values[24])

str

In [73]:
pvDaysFinalFinal

array([[   0.   ,    0.   ,    0.   , ...,    0.   ,    0.   ,    0.   ],
       [   0.   ,   82.85 ,  443.984, ...,  458.198, 1020.896, 1551.23 ],
       [2152.544, 2266.974, 2462.567, ..., 1780.11 , 2339.352, 2175.408],
       ...,
       [2176.275, 2554.959, 2764.791, ..., 2541.2  , 2857.429, 2264.494],
       [1903.385, 1351.628,  582.222, ...,  558.068,   99.931,    0.   ],
       [   0.   ,    0.   ,    0.   , ...,    0.   ,    0.   ,    0.   ]])

In [76]:
# Battery maximum state of charge
soc_max = 77000

# Battey minimum state of charge (50% max capacity)
soc_min = soc_max * 0.5

# Inverter efficiency
inv_eff = 0.9

# Battery efficiency
bat_eff = 0.9

# Empty array of days vs hours for Photo Voltaic Energy generation
pve = np.zeros((days, K))

#Empty array of days vs hours for battery State Of Charge
soc = np.zeros((days, K+1))

#Initial state of charge is the capacity minimum
soc[0, 0] = ((soc_max - soc_min)/(2 + soc_min))

#Empty array of zeroes for excess energy generation
pve_exs = np.zeros((days, K))

#Empty array of zeroes for energy deficit
pve_def = np.zeros((days, K))

#
for x in range(1, days, 1):
    for y in range(1, K, 1):
        if (pvo[y, x]-((1/inv_eff)*demand.iloc[y, 0])) > 0:
            pve[x, y] = pvo[y, x] - ((1/inv_eff)*demand.iloc[y, 0])
        elif (pvo[y, x]-((1/inv_eff)*demand.iloc[y, 0])) < 0:
            pve[x, y] = (pvo[y, x] - ((1/inv_eff)*demand.iloc[y, 0]))*(1/bat_eff)
        else:
            pve[x, y] = 0
        
        soc[x, y+1] = soc[x, y] + pve[x, y]
        if soc[x, y+1] >= soc_max:
            pve_exs[x, y] = soc[x, y+1] - soc_max
            soc[x, y+1] = soc_max
        elif soc[x, y+1] <= soc_min:
            pve_def[x, y] = soc_min - soc[x, y+1]
            soc[x, y+1] = soc_min
        else:
            pass
        
    if x != days:
        soc[x+1, 1] = soc[x, K+1]


IndexingError: Too many indexers

In [45]:
# Battery maximum state of charge
soc_max = 77000

# Battey minimum state of charge (50% max capacity)
soc_min = soc_max * 0.5

# Inverter efficiency
inv_eff = 0.9

# Battery efficiency
bat_eff = 0.9

# Empty array of days vs hours for Photo Voltaic Energy generation
pve = np.zeros((days, K))

#Empty array of days vs hours for battery State Of Charge
soc = np.zeros((days, K+1))

#Initial state of charge is the capacity minimum
soc[0, 0] = ((soc_max - soc_min)/(2 + soc_min))

#Empty array of zeroes for excess energy generation
pve_exs = np.zeros((days, K))

#Empty array of zeroes for energy deficit
pve_def = np.zeros((days, K))

#
for x in range(1, days, 1):
    for y in range(1, K, 1):
        if (pvDaysFinal[y, x]-((1/inv_eff)*hourly_load[y])) > 0:
            pve[x, y] = pvDaysFinal[y, x] - ((1/inv_eff)*hourly_load[y])
        elif (pvDaysFinal[y, x]-((1/inv_eff)*hourly_load[y])) < 0:
            pve[x, y] = (pvDaysFinal[y, x] - ((1/inv_eff)*hourly_load[y]))*(1/bat_eff)
        else:
            pve[x, y] = 0
        
        soc[x, y+1] = soc[x, y] + pve[x, y]
        if soc[x, y+1] >= soc_max:
            pve_exs[x, y] = soc[x, y+1] - soc_max
            soc[x, y+1] = soc_max
        elif soc[x, y+1] <= soc_min:
            pve_def[x, y] = soc_min - soc[x, y+1]
            soc[x, y+1] = soc_min
        else:
            pass
        
    if x != days:
        soc[x+1, 1] = soc[x, K+1]


IndexError: too many indices for array

In [41]:
soc.shape

(365, 25)

In [10]:
print(num)

20


In [12]:
for i in range(1, num, 1):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
